In [ ]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

In [ ]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(
    name='quick-starts-ws-254578',
    subscription_id='a24a24d5-8d87-4c8a-99b6-91ed2d2df51f',
    resource_group='aml-quickstarts-254578',
)
exp = Experiment(workspace=ws, name="udacity-project")


In [30]:
from azureml.core import Workspace, Experiment

ws= Workspace.get(
name='quick-starts-ws-254650',
subscription_id='a24a24d5-8d87-4c8a-99b6-91ed2d2df51f',
resource_group='aml-quickstarts-254650',  
)
exp = Experiment(workspace=ws, name="udacity-project")
run = exp.start_logging()

In [31]:
from azureml.core.compute import ComputeTarget, AmlCompute

cluster_name = "cluster-exerc"


# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###

compute_comfig = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2', max_nodes=4)
aml_compute = ComputeTarget.create(ws, cluster_name, compute_comfig)
#aml_Compute.wait_for_completion(show_output=True)

In [53]:
from azureml.widgets import RunDetails
from azureml.core import Workspace,Experiment,Run
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
#ps = ### YOUR CODE HERE ###
ps = RandomParameterSampling({
    "--C" : choice(0.01, 0.1, 1),
    "--max_iter" : choice(20,50,100,150,200)
})

# Specify a Policy
#policy = ### YOUR CODE HERE ###
policy = BanditPolicy(evaluation_interval=1, slack_factor=0.2, slack_amount=None, delay_evaluation=0)
if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
#src = ### YOUR CODE HERE ###
src = ScriptRunConfig(
    source_directory='.',
    script='train.py',
    compute_target=aml_compute,
    environment=sklearn_env,
)
# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
#hyperdrive_config = ### YOUR CODE HERE ###
hyperdrive_config = HyperDriveConfig(run_config=src,
                                     hyperparameter_sampling=ps,
                                     policy=policy,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs = 4,
                                     max_concurrent_runs=4)

In [54]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hyperdrive_run = exp.submit(config=hyperdrive_config)
#hyperdrive_run.wait_for_completion()
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [57]:
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [58]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
#hyperdrive_run.list()
#print("best run file name", best_run.get_file_names())
print(best_run)

Run(Experiment: udacity-project,
Id: HD_3bd2906c-a5c2-4c2a-a746-1ba13bc62519_3,
Type: azureml.scriptrun,
Status: Completed)


In [60]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
#list(exp.get_runs())
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
print('Best Run ID:', best_run.id)
print("\n")
print("best run file name", best_run.get_file_names())
print('\n best_run_metric:', best_run_metrics)




Best Run ID: HD_3bd2906c-a5c2-4c2a-a746-1ba13bc62519_3


best run file name ['logs/azureml/dataprep/0/backgroundProcess.log', 'logs/azureml/dataprep/0/backgroundProcess_Telemetry.log', 'logs/azureml/dataprep/0/rslex.log.2024-03-12-17', 'system_logs/cs_capability/cs-capability.log', 'system_logs/hosttools_capability/hosttools-capability.log', 'system_logs/lifecycler/execution-wrapper.log', 'system_logs/lifecycler/lifecycler.log', 'system_logs/metrics_capability/metrics-capability.log', 'system_logs/snapshot_capability/snapshot-capability.log', 'user_logs/std_log.txt']

 best_run_metric: {'Regularization Strength:': 0.01, 'Max iterations:': 100, 'Accuracy': 0.907950637264819}


In [92]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
ds = TabularDatasetFactory.from_delimited_files("https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")
#print(ds.to_pandas_dataframe())
print(ds)

TabularDataset
{
  "source": [
    "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
  ],
  "definition": [
    "GetFiles",
    "ParseDelimited",
    "DropColumns",
    "SetColumnTypes"
  ]
}


In [101]:
from train import clean_data
from sklearn.model_selection import train_test_split
from azureml.core import Dataset
import pandas as pd
# Use the clean_data function to clean your data.
x, y = clean_data(ds)
print(type(x))
print(type(y))

all_data = pd.concat([x, y], axis=1)
train_data, test_data = train_test_split(all_data, test_size=0.2, random_state=66)
train_data.to_csv('training/training_data.csv')
#X_train, X_test, Y_train, Y_test =train_test_split(x,y, test_size=0.2, random_state=42)

datas = ws.get_default_datastore()
datas.upload(src_dir='training', target_path='training_data')

train_ds = Dataset.Tabular.from_delimited_files(path=[(datas,('training_data/training_data.csv'))])
#train_data = pd.concat([X_train, Y_train], axis=1)
#test_data = pd.concat([X_test, Y_test], axis=1)

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>
Uploading an estimated of 1 files
Target already exists. Skipping upload for training_data/training_data.csv
Uploaded 0 files


In [102]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=train_ds,
    label_column_name='y',
    n_cross_validations=5,
    compute_target=aml_compute,
    enable_early_stopping=True,
    enable_onnx_compatible_models=True
)

In [103]:
# Submit your automl run

### YOUR CODE HERE ###
#automl_exp = ws.experiments
automl_run = exp.submit(automl_config, show_output = True)

Submitting remote run.
No run_configuration provided, running on cluster-exerc with default configuration
Running on remote compute: cluster-exerc


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_4174da30-b117-4239-9d91-d6c995dc56f0,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+--------------------------------------+
|Size of the smallest class    |Name/Label of the smallest class|Nu

In [105]:
from azureml.automl.runtime.onnx_convert import OnnxConverter
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
automl_run.get_metrics()
#automl_run.get_tags()
automl_best_run_onnx, automl_fitted_model_onnx = automl_run.get_output(return_onnx_model=True)
OnnxConverter.save_onnx_model(automl_fitted_model_onnx, './outputs/best_model_onnx')

In [106]:
print(automl_run.get_metrics())

{'experiment_status': ['DatasetEvaluation', 'FeaturesGeneration', 'DatasetFeaturization', 'DatasetFeaturizationCompleted', 'DatasetBalancing', 'DatasetCrossValidationSplit', 'ModelSelection', 'BestRunExplainModel', 'ModelExplanationDataSetSetup', 'PickSurrogateModel', 'EngineeredFeatureExplanations', 'EngineeredFeatureExplanations', 'RawFeaturesExplanations', 'RawFeaturesExplanations', 'BestRunExplainModel'], 'experiment_status_description': ['Gathering dataset statistics.', 'Generating features for the dataset.', 'Beginning to fit featurizers and featurize the dataset.', 'Completed fit featurizers and featurizing the dataset.', 'Performing class balancing sweeping', 'Generating individually featurized CV splits.', 'Beginning model selection.', 'Best run model explanations started', 'Model explanations data setup completed', 'Choosing LightGBM as the surrogate model for explanations', 'Computation of engineered features started', 'Computation of engineered features completed', 'Computa